In [16]:
from scipy import stats
import time
# import requests
from bs4 import BeautifulSoup
import threading
import pandas as pd
import tqdm
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler, PowerTransformer
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle
import sys

   
sys.path.append("/home/td/Documents/sports_predictor")
sys.path.append("/home/td/Documents/sports_predictor/nba")

from process_data import DataManager

base_url = 'https://www.basketball-reference.com/'
day_scores_base_url = 'https://www.basketball-reference.com/boxscores/?month={month}&day={day}&year={year}'
data_path = r'/media/td/Samsung_T5/sports/nba'
# data_path = r'C:\Users\TristanDelforge\Documents\sports_predictor\nba'
db_name = 'nba_db'
box_score_link_table_name = 'boxscore_links'

box_score_details_table_name = 'boxscore_details'
player_detail_table_name = 'player_details'
box_score_details_table_name_sample = 'boxscore_details_sample'
player_detail_table_name_sample = 'player_details_sample'
processed_team_data_table_name = 'processed_team_data'
processed_player_data_table_name = 'processed_player_data'

processed_team_data_table_name = 'processed_team_data'
processed_player_data_table_name = 'processed_player_data'
aggregated_player_data_table_name = 'aggregated_player_data'
combined_feature_file_data_table_name = 'combined_feature_file'
past_n_game_dataset_table_name = 'past_n_game_dataset'
target = 'win'

date_record_pickle_file_name = 'scraped_dates'
box_score_record_pickle_file_name = 'scraped_games'
max_tries = 5
file_lock = threading.Lock()

starting_rating = 1000
rating_k_factor = 100
rating_floor = 100
rating_ceiling = 10000
rating_d = 1000
k_min_sensitivity = 1

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [2]:
team_data = pd.read_csv('{data_path}/{db_name}.csv'.format(data_path=data_path,db_name=box_score_details_table_name),sep='|', low_memory=False)
player_data = pd.read_csv('{data_path}/{db_name}.csv'.format(data_path=data_path,db_name=player_detail_table_name), sep='|', low_memory=False)



In [3]:
team_data.shape

(58158, 45)

In [4]:
team_data.head()

,team_tag,team_link,team_name,opponent_tag,opponent_link,opponent_name,location,win,year,month,day,mp,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,fta,ft_pct,orb,drb,trb,ast,stl,blk,tov,pf,pts,plus_minus,ts_pct,efg_pct,fg3a_per_fga_pct,fta_per_fga_pct,orb_pct,drb_pct,trb_pct,ast_pct,stl_pct,blk_pct,tov_pct,usg_pct,off_rtg,def_rtg
0,por,https://www.basketball-reference.com//teams/PO...,Portland Trail Blazers,van,https://www.basketball-reference.com//teams/VA...,Vancouver Grizzlies,NaN,1,1996,11,1,240,45,86,0.523,7,17,0.412,17,27,0.630,22,32,54,30,12,7,19,20,114,NaN,0.582,0.564,0.198,0.314,55.0,78.0,66.7,66.7,13.2,11.1,16.3,100.0,125.8,93.8
1,van,https://www.basketball-reference.com//teams/VA...,Vancouver Grizzlies,por,https://www.basketball-reference.com//teams/PO...,Portland Trail Blazers,NaN,0,1996,11,1,240,34,77,0.442,5,14,0.357,12,19,0.632,9,18,27,23,7,5,15,21,85,NaN,0.498,0.474,0.182,0.247,22.0,45.0,33.3,67.6,7.7,7.2,14.9,100.0,93.8,125.8
2,sea,https://www.basketball-reference.com//teams/SE...,Seattle SuperSonics,uta,https://www.basketball-reference.com//teams/UT...,Utah Jazz,NaN,0,1996,11,1,240,30,75,0.400,5,18,0.278,26,34,0.765,12,34,46,16,6,6,12,26,91,NaN,0.506,0.433,0.240,0.453,27.3,75.6,51.7,53.3,6.8,10.0,11.8,100.0,103.3,112.4
3,uta,https://www.basketball-reference.com//teams/UT...,Utah Jazz,sea,https://www.basketball-reference.com//teams/SE...,Seattle SuperSonics,NaN,1,1996,11,1,240,34,78,0.436,8,18,0.444,23,33,0.697,11,32,43,26,7,6,9,27,99,NaN,0.535,0.487,0.231,0.423,24.4,72.7,48.3,76.5,7.9,10.5,8.9,100.0,112.4,103.3
4,nyk,https://www.basketball-reference.com//teams/NY...,New York Knicks,tor,https://www.basketball-reference.com//teams/TO...,Toronto Raptors,NaN,1,1996,11,1,240,33,68,0.485,4,11,0.364,37,51,0.725,11,33,44,23,15,5,24,29,107,NaN,0.592,0.515,0.162,0.750,29.7,75.0,54.3,69.7,14.7,9.4,21.0,100.0,104.7,96.9


In [6]:
player_data.shape

(712166, 48)

In [7]:
player_data.head()

,ast,ast_pct,blk,blk_pct,day,def_rtg,drb,drb_pct,efg_pct,fg,fg3,fg3_pct,fg3a,fg3a_per_fga_pct,fg_pct,fga,ft,ft_pct,fta,fta_per_fga_pct,location,month,mp,off_rtg,opponent_link,opponent_name,opponent_tag,orb,orb_pct,pf,player_link,player_name,plus_minus,pts,reason,stl,stl_pct,team_link,team_name,team_tag,tov,tov_pct,trb,trb_pct,ts_pct,usg_pct,win,year
0,8.0,41.7,0.0,0.0,1,90.0,4.0,16.1,0.731,8.0,3.0,0.6,5.0,0.385,0.615,13.0,1.0,0.500,2.0,0.154,NaN,11,29:19,133.0,https://www.basketball-reference.com//teams/VA...,Vancouver Grizzlies,van,3.0,12.4,2.0,https://www.basketball-reference.com//players/...,Kenny Anderson,20.0,20.0,NaN,3.0,5.5,https://www.basketball-reference.com//teams/PO...,Portland Trail Blazers,por,4.0,22.4,7.0,14.3,0.720,25.3,1,1996
1,1.0,5.2,1.0,2.6,1,101.0,2.0,8.1,0.600,8.0,2.0,0.4,5.0,0.333,0.533,15.0,4.0,0.667,6.0,0.400,NaN,11,28:38,130.0,https://www.basketball-reference.com//teams/VA...,Vancouver Grizzlies,van,0.0,0.0,2.0,https://www.basketball-reference.com//players/...,Clifford Robinson,15.0,22.0,NaN,0.0,0.0,https://www.basketball-reference.com//teams/PO...,Portland Trail Blazers,por,2.0,10.2,2.0,4.1,0.624,27.8,1,1996
2,1.0,5.2,2.0,5.9,1,98.0,2.0,9.0,0.714,5.0,0.0,NaN,0.0,0.000,0.714,7.0,0.0,0.000,2.0,0.286,NaN,11,25:40,122.0,https://www.basketball-reference.com//teams/VA...,Vancouver Grizzlies,van,2.0,9.2,2.0,https://www.basketball-reference.com//players/...,Rasheed Wallace,24.0,10.0,NaN,0.0,0.0,https://www.basketball-reference.com//teams/PO...,Portland Trail Blazers,por,1.0,11.3,4.0,9.1,0.635,14.0,1,1996
3,4.0,19.5,1.0,3.2,1,88.0,1.0,4.9,0.400,2.0,0.0,NaN,0.0,0.000,0.400,5.0,4.0,0.800,5.0,1.000,NaN,11,24:17,160.0,https://www.basketball-reference.com//teams/VA...,Vancouver Grizzlies,van,0.0,0.0,0.0,https://www.basketball-reference.com//players/...,Aaron McKie,22.0,8.0,NaN,3.0,6.6,https://www.basketball-reference.com//teams/PO...,Portland Trail Blazers,por,0.0,0.0,1.0,2.5,0.556,12.3,1,1996
4,2.0,10.8,3.0,9.9,1,83.0,7.0,35.6,0.600,3.0,0.0,0.0,1.0,0.200,0.600,5.0,3.0,0.429,7.0,1.400,NaN,11,22:46,69.0,https://www.basketball-reference.com//teams/VA...,Vancouver Grizzlies,van,3.0,15.7,3.0,https://www.basketball-reference.com//players/...,Arvydas Sabonis,18.0,9.0,NaN,1.0,2.3,https://www.basketball-reference.com//teams/PO...,Portland Trail Blazers,por,7.0,46.4,10.0,25.8,0.557,26.9,1,1996


In [10]:
team_data_recent = team_data[team_data['year'] >= 2014]
player_data_recent = player_data[player_data['year'] >= 2014]

In [11]:
team_data_recent.shape, player_data_recent.shape

((14806, 45), (188227, 48))

In [14]:
team_data_recent.to_csv('{data_path}/{db_name}.csv'.format(data_path=data_path,db_name=box_score_details_table_name_sample),sep='|', index=False)
player_data_recent.to_csv('{data_path}/{db_name}.csv'.format(data_path=data_path,db_name=player_detail_table_name_sample), sep='|', index=False)

In [15]:
def get_processed_data():
    team_df = pd.read_csv(f'{data_path}/{processed_team_data_table_name}.csv', sep = '|', low_memory=False)
    player_df = pd.read_csv(f'{data_path}/{processed_player_data_table_name}.csv', sep = '|', low_memory=False)
    return team_df, player_df

In [17]:
team_df, player_df = get_processed_data()

In [18]:
team_df.head()

,Unnamed: 0,team_tag,team_link,team_name,opponent_tag,opponent_link,opponent_name,location,win,year,month,day,mp,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,fta,ft_pct,orb,drb,trb,ast,stl,blk,tov,pf,pts,plus_minus,ts_pct,efg_pct,fg3a_per_fga_pct,fta_per_fga_pct,orb_pct,drb_pct,trb_pct,ast_pct,stl_pct,blk_pct,tov_pct,usg_pct,off_rtg,def_rtg,home,year_str,month_str,day_str,date_str,date_dt,days_since_last_match,team_pregame_rating_0,team_postgame_rating_0,team_pregame_rating_1,team_postgame_rating_1
0,0,sac,https://www.basketball-reference.com//teams/SA...,Sacramento Kings,tor,https://www.basketball-reference.com//teams/TO...,Toronto Raptors,"Scotiabank Arena, Toronto, Canada",0,2019,1,22,0.0,41,91,0.451,7,21,0.333,16,22,0.727,10,33,43,29,8,7,14,27,105,NaN,0.521,0.489,0.231,0.242,22.7,73.3,48.3,70.7,7.7,13.7,12.2,100.0,100.9,115.3,0,2019,1,22,2019-01-22,2019-01-22,NaN,1000.0,950.0,1000.0,950.0
1,1,tor,https://www.basketball-reference.com//teams/TO...,Toronto Raptors,sac,https://www.basketball-reference.com//teams/SA...,Sacramento Kings,"Scotiabank Arena, Toronto, Canada",1,2019,1,22,0.0,40,91,0.440,16,40,0.400,24,35,0.686,12,34,46,25,7,5,16,21,120,NaN,0.564,0.527,0.440,0.385,26.7,77.3,51.7,62.5,6.7,7.1,13.1,100.0,115.3,100.9,1,2019,1,22,2019-01-22,2019-01-22,NaN,1000.0,1050.0,1000.0,1050.0
2,2,min,https://www.basketball-reference.com//teams/MI...,Minnesota Timberwolves,pho,https://www.basketball-reference.com//teams/PH...,Phoenix Suns,"Talking Stick Resort Arena, Phoenix, Arizona",1,2019,1,22,0.0,45,103,0.437,15,42,0.357,13,19,0.684,20,44,64,30,7,1,14,18,118,NaN,0.530,0.510,0.408,0.184,40.8,89.8,65.3,66.7,7.0,1.9,11.2,100.0,117.4,90.5,0,2019,1,22,2019-01-22,2019-01-22,NaN,1000.0,1050.0,1000.0,1050.0
3,3,pho,https://www.basketball-reference.com//teams/PH...,Phoenix Suns,min,https://www.basketball-reference.com//teams/MI...,Minnesota Timberwolves,"Talking Stick Resort Arena, Phoenix, Arizona",0,2019,1,22,0.0,37,87,0.425,7,34,0.206,10,18,0.556,5,29,34,23,6,9,13,17,91,NaN,0.479,0.466,0.391,0.207,10.2,59.2,34.7,62.2,6.0,14.8,12.0,100.0,90.5,117.4,1,2019,1,22,2019-01-22,2019-01-22,NaN,1000.0,950.0,1000.0,950.0
4,4,por,https://www.basketball-reference.com//teams/PO...,Portland Trail Blazers,okc,https://www.basketball-reference.com//teams/OK...,Oklahoma City Thunder,"Chesapeake Energy Arena, Oklahoma City, Oklahoma",0,2019,1,22,0.0,43,95,0.453,10,36,0.278,18,22,0.818,20,32,52,21,7,3,19,20,114,NaN,0.545,0.505,0.379,0.232,41.7,78.0,58.4,48.8,7.1,4.8,15.4,100.0,115.2,124.3,0,2019,1,22,2019-01-22,2019-01-22,NaN,1000.0,950.0,1000.0,950.0
